In [1]:
%env MUJOCO_GL=egl
import gym
import os
import json
import math
import heapq
import random
import argparse
from pathlib import Path
import numpy as np
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
from tqdm import tqdm

def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

env: MUJOCO_GL=egl


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
from dreamer import Dreamer, make_env
import tools
import argparse
from ruamel.yaml import YAML
yaml = YAML(typ='safe', pure=True)
import torch
from parallel import Damy

def recursive_update(base, update):
    for key, value in update.items():
        if isinstance(value, dict) and key in base:
            recursive_update(base[key], value)
        else:
            base[key] = value

def load_dreamer(task, ckpt_path, device):
    print("load_dreamer")
    config_path = '/scorpio/home/yubei-stu-2/agent_dreamerv3_torch/configs.yaml' # server
    with open(config_path, 'r') as file:
        config = yaml.load(file)
    defaults = {}
    for name in ['defaults', 'memorymaze']: # trigger, plus re.match(cnn_key)
        recursive_update(defaults, config[name])
    
    config = argparse.Namespace(**defaults)
    config.device = device
    config.task = task
    env = Damy(make_env(config, "train", 0))
    acts = env.action_space
    config.num_actions = acts.n if hasattr(acts, "n") else acts.shape[0] # trigger

    agent = Dreamer(
        env.observation_space,
        env.action_space,
        config,
        None, # no logger, change __init__
        None,
    )
    agent.requires_grad_(requires_grad=False)
    checkpoint = torch.load(ckpt_path,weights_only=False)
    agent.load_state_dict(checkpoint["agent_state_dict"],strict=False)
    # tools.recursively_load_optim_state_dict(agent, checkpoint["optims_state_dict"])
    agent._should_pretrain._once = False
    return agent, env

ckpt_path = '/scorpio/home/yubei-stu-2/agent_dreamerv3_torch/agent_maze2.pt'
task = 'memorymaze_9x9'
agent, env = load_dreamer(task, ckpt_path, device='cuda:0')

load_dreamer
Encoder CNN shapes: {'image': (64, 64, 3)}
Encoder MLP shapes: {}
Decoder CNN shapes: {'image': (64, 64, 3)}
Decoder MLP shapes: {}


/scorpio/home/yubei-stu-2/agent_dreamerv3_torch/tools.py:747: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self._scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


Optimizer model_opt has 15687811 variables.
Optimizer actor_opt has 1053702 variables.
Optimizer value_opt has 1181439 variables.


In [ ]:
# agent policy pattern
agent_state = None
action, agent_state = agent(obs, done, agent_state, training=False)

In [6]:
import memory_maze 
env = gym.make('MemoryMaze-9x9-ExtraObs-v0')
print(env.observation_space)

def obs_dreamer(inp_obs, done, is_first):
    obs = inp_obs.copy()
    obs["is_first"]    = np.array([bool(is_first)], dtype=bool)
    obs["is_last"]     = np.array([bool(done)], dtype=bool)
    obs["is_terminal"] = np.array([False], dtype=bool)
    for k in obs.keys():
        obs[k] = obs[k][None, :]
    return obs

seq_len = 30
keys = ["obs", "reward", "done", "action"]
obs_key = ['agent_pos', 'agent_dir']
frames = []     
seq = {k: [] for k in keys}

obs = env.reset()
agent_state = None
agent_obs = obs_dreamer(obs, False, True)
done = False

for _ in tqdm(range(seq_len)):
    for _ in range(4):
        
        action, agent_state = agent(agent_obs, done, agent_state, training=False)
        action = int(action['action'].argmax())
        obs, reward, done, info = env.step(action) # a_t -> o_t
        frames.append(obs['image'])
        agent_obs = obs_dreamer(obs, done, False) # add dim after append
        obs = np.concatenate([obs[k] for k in obs_key], axis=-1)
        # seq['obs'].append(obs)
        # seq['reward'].append(reward)
        # seq['done'].append(done)
        # seq['action'].append(action)
        
        if done:
            env.reset()
            break
display_video(frames, framerate=20)

/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (9, 9)
  logger.warn(
/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (3, 2)
  logger.warn(
/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `En

Dict('agent_dir': Box(-inf, inf, (2,), float64), 'agent_pos': Box(-inf, inf, (2,), float64), 'image': Box(0, 255, (64, 64, 3), uint8), 'maze_layout': Box(0, 1, (9, 9), uint8), 'target_color': Box(-inf, inf, (3,), float64), 'target_pos': Box(-inf, inf, (2,), float64), 'target_vec': Box(-inf, inf, (2,), float64), 'targets_pos': Box(-inf, inf, (3, 2), float64), 'targets_vec': Box(-inf, inf, (3, 2), float64))


/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'dict'>`
  logger.warn(
  0%|          | 0/30 [00:00<?, ?it/s]/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
100%|██████████| 30/30 [00:06<00:00,  4.65it/s]


In [13]:
# seq gen for panda
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR) 

task = 'Maze' # 'Pong-v5' # "spin"
act_mode = "expert" # ['zero', 'random', 'expert']
train_param = [1000, 1000, '/scorpio/home/yubei-stu-2/dreamerv3_torch_ver/data/' + f"seq-{task}-{act_mode}.npz"]
test_param = [200, 100, '/scorpio/home/yubei-stu-2/dreamerv3_torch_ver/data/' + f"seq-{task}-{act_mode}-test.npz"]
keys = ["obs", "reward", "done", "action"]
obs_key = ['agent_pos', 'agent_dir']

import memory_maze 
env = gym.make('MemoryMaze-9x9-ExtraObs-v0')

def obs_dreamer(inp_obs, done, is_first):
    obs = inp_obs.copy()
    obs["is_first"]    = np.array([bool(is_first)], dtype=bool)
    obs["is_last"]     = np.array([bool(done)], dtype=bool)
    obs["is_terminal"] = np.array([False], dtype=bool)
    for k in obs.keys():
        obs[k] = obs[k][None, :]
    return obs

def seq_gen(seq_cnt, seq_len, savepath):
    min_seq = seq_len
    data = {k: [] for k in keys}
    pbar = tqdm(total=seq_cnt)
    while len(data['obs']) < seq_cnt:
        seq = {k: [] for k in keys}
        obs = env.reset()
        agent_state = None
        agent_obs = obs_dreamer(obs, False, True)
        done = False
        for _ in range(seq_len):
            action, agent_state = agent(agent_obs, done, agent_state, training=False)
            action = int(action['action'].argmax())
            obs, reward, done, info = env.step(action) # a_t -> o_t
            agent_obs = obs_dreamer(obs, done, False) # add dim after append
            obs = np.concatenate([obs[k] for k in obs_key], axis=-1)
            seq['obs'].append(obs)
            seq['reward'].append(reward)
            seq['done'].append(done)
            seq['action'].append(action)
            if done:
                break
        if len(seq['obs']) < seq_len: # 0.6 * seq_len:
            print('skip ', len(seq['obs']))
            continue
        if len(seq['obs']) < min_seq:
            min_seq = len(seq['obs'])
        for k in keys:
            data[k].append(seq[k])
        pbar.n = len(data['obs'])
        pbar.refresh()

    for key in data.keys():
        data[key] = [x[:min_seq] for x in data[key]]
    data = {k: np.array(data[k]) for k in keys}
    for key in data.keys():
        print(key, data[key].shape, np.isinf(data[key]).any(), np.isnan(data[key]).any())
    metadata = {}
    np.savez_compressed(savepath, **data, metadata=metadata)
    print('saved ', savepath)

seq_gen(*test_param)
# seq_gen(*train_param)

KeyboardInterrupt: 

In [13]:
# fast version
# seq gen for panda
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR) 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

task = 'Maze' # 'Pong-v5' # "spin"
act_mode = "expert" # ['zero', 'random', 'expert']
train_param = [1000, 1000, '/scorpio/home/yubei-stu-2/dreamerv3_torch_ver/data/' + f"seq-{task}-{act_mode}.npz"]
test_param = [200, 100, '/scorpio/home/yubei-stu-2/dreamerv3_torch_ver/data/' + f"seq-{task}-{act_mode}-test.npz"]
keys = ["obs", "reward", "done", "action"]
obs_key = ['agent_pos', 'agent_dir']

import memory_maze 
# env = gym.make('MemoryMaze-9x9-ExtraObs-v0')
B = 25
envs = [gym.make('MemoryMaze-9x9-ExtraObs-v0') for _ in range(B)]

def obs_dreamer(inp_obs, done, is_first):
    obs = inp_obs.copy()
    obs["is_first"]    = np.array(is_first, dtype=bool)
    obs["is_last"]     = np.array(done, dtype=bool)
    obs["is_terminal"] = np.zeros_like(obs["is_last"])
    # for k in obs.keys():
    #     obs[k] = obs[k][None, :]
    return obs

def seq_gen(seq_cnt, seq_len, savepath):
    min_seq = seq_len
    data = {k: [] for k in keys}
    while len(data['obs']) < seq_cnt:
        print(len(data['obs']))
        seq = {k: [] for k in keys}
        obs =[env.reset() for env in envs]
        obs = {k: np.stack([o[k] for o in obs]) for k in obs[0]}
        agent_state = None
        done = np.zeros(B, dtype=bool)    # (B,)
        is_first = np.ones(B, dtype=bool) # (B,)
        agent_obs = obs_dreamer(obs, done, is_first)
        for _ in range(seq_len):
            action, agent_state = agent(agent_obs, done, agent_state, training=False)
            action = action['action'].argmax(axis=-1).cpu().numpy().astype(np.int64)
            result = [env.step(act) for env, act in zip(envs, action)]
            obs, reward, done = zip(*[p[:3] for p in result])
            obs = {k: np.stack([o[k] for o in obs]) for k in obs[0]}
            #obs, reward, done, info = env.step(action) # a_t -> o_t
            agent_obs = obs_dreamer(obs, done, np.zeros(B, dtype=bool)) # add dim after append
            obs = np.concatenate([obs[k] for k in obs_key], axis=-1)
            reward = np.array(reward)
            done = np.array(done, dtype=bool)
            # print(done, reward)
            # print('obs', obs.shape) # [B, 4]
            # print('reward', reward) # [B, 1]
            seq['obs'].append(obs)
            seq['reward'].append(reward)
            seq['done'].append(done)
            seq['action'].append(action)
            if done.any():
                print(done)
                break
        if len(seq['obs']) < seq_len: # 0.6 * seq_len:
            print('skip ', len(seq['obs']))
            continue

        for k in keys:
            seq[k] = np.swapaxes(np.array(seq[k]), 0, 1) # [t, b, d] -> [b, t, d]
            for i in range(B):
                data[k].append(seq[k][i])

    data = {k: np.array(data[k]) for k in keys}
    for key in data.keys():
        print(key, data[key].shape, np.isinf(data[key]).any(), np.isnan(data[key]).any())
    metadata = {}
    np.savez_compressed(savepath, **data, metadata=metadata)
    print('saved ', savepath)

seq_gen(*test_param)
# seq_gen(*train_param)

/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (9, 9)
  logger.warn(
/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:31: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (3, 2)
  logger.warn(


0


/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/scorpio/home/yubei-stu-2/miniconda3/envs/maze/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'dict'>`
  logger.warn(
  0%|          | 0/200 [04:42<?, ?it/s]


25
50
75
100
125
150
175
obs (200, 100, 4) False False
reward (200, 100) False False
done (200, 100) False False
action (200, 100) False False
saved  /scorpio/home/yubei-stu-2/dreamerv3_torch_ver/data/seq-Maze-expert-test.npz
